<a href="https://colab.research.google.com/github/vinay235/llm-experiments/blob/main/Snowflake_Agents_using_Autogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## !pip install snowflake-snowpark-python pyautogen pandas

## Import all libraries

In [ ]:
# Print the version of Snowpark we are using
from snowflake.snowpark import Session
import pandas as pd
import autogen

## Open a Snowflake Session

In [ ]:

connection_parameters = {
    "account": "",
    "user": "",
    "password": "",
    "warehouse": "COMPUTE_WH",
    "database": "DEMO_DB",
    "schema": "DEMO_SCH"
}

session = Session.builder.configs(connection_parameters).create()
print("Current role: " + session.get_current_role() + ", Current schema: " + session.get_fully_qualified_current_schema())

Current role: "ACCOUNTADMIN", Current schema: "DEMO_DB"."DEMO_SCH"


## Enter OpenAI API keys

In [ ]:
config_list = [
    {
        'model': 'gpt-3.5-turbo-16k-0613',#'gpt-3.5-turbo-0613',
        'api_key': '',
    }
]

## Create a function to run query on Snowflake

In [ ]:
def run_sql(query):
    res = session.sql(f"{query}").collect()
    df = pd.DataFrame(data = res)
    if(len(df) > 0):
        return df
    else:
        return "NO ROWS RETURNED"
    return df

## Enter the LLM config details needed for the Agents

In [ ]:
llm_config_code = {
    "temperature": 0,
    "request_timeout": 600,
    #"seed": 43,
    "functions": [
        {
            "name" : "run_sql",
            "description" : "running SQL query on Snowflake",
            "parameters" : {
                "type" : "object",
                "properties": {
                        "query": {
                            "type": "string",
                            "description": "SQL query",
                        }
                    },
                    "required": ["query"],
                },
            },
         ],
    "config_list": config_list
    }


## Configure the Agents and the GroupChat Manager

In [ ]:
user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
)
engineer = autogen.AssistantAgent(
    name="SnowEngineer",
    llm_config=llm_config_code,
    system_message='''SnowEngineer. You write Snowflake SQL code to solve tasks. You are also capable of writing Snowflake Stored procedures in Python.
    Wrap the code in a code block that specifies the script type. The user can't modify your code. The tables can contain large data. DO not query all rows. ACT RESPONSIBLY.
    You are responsible to check the metadata of the tables by using the SHOW TABLES command and finding the table and columns that is needed.
    Use DESCRIBE TABLE to understand its data and columns. Provide a sample of 10 rows as output along with the query in code block.
    Do not ask others to copy and paste the result.
Check the execution result returned by the SnowSQLAgent.
You will suggest SnowSQLAgent to search the web and get the latest info ALWAYS.
If the result indicates there is an error, fix the error and output the code again.
Suggest the full code instead of partial code or code changes.
If the error can't be fixed or if the task is not solved even after the code is executed successfully,
analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.

''',
)

planner = autogen.AssistantAgent(
    name="SnowPlanner",
    system_message="""SnowPlanner. The plan will be given by SnowEngineer. You will break down the tasks and send one query each to SnowSQLAgent.
You will not suggest anything. You will only take SnowEngineer's recommendation and send ONE query only to SnowSQLAgent.
 Don't include multiple SQL queries in one response. If you have many queries, break down the steps and send it one by one to the SnowSQLAgent.
  REMEMBER, you can SEND only ONE query at once.
""",
    llm_config=llm_config_code,
)
executor = autogen.UserProxyAgent(
    name="SnowSQLAgent",
    system_message="""SnowSQLAgent. Execute the code written by the SnowEngineer.
    You have the tools and functions to search the web to provide the most up-to-date information.
    You will search, scrape, and summarise the information using the tools provided and report back to SnowEngineer to modify the queries.
    You will NOT suggest anything apart from what you find from the tools.""",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "sql"},
    function_map={
                "run_sql": run_sql,
                "search": search,
                "scrape": scrape,
            },
)

groupchat = autogen.GroupChat(agents=[user_proxy, engineer, planner, executor], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config_code)

## Run basic Create and Insert commands

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="""
Create two tables Orders and Customers with 5 columns each. You can decide the appropriate columns needed and its datatype.
Insert 10 rows in each of the tables.
""",
)

Admin (to chat_manager):


Create two tables Orders and Customers with 5 columns each. You can decide the appropriate columns needed and its datatype.
Insert 10 rows in each of the tables. 


--------------------------------------------------------------------------------
SnowEngineer (to chat_manager):

***** Suggested function Call: run_sql *****
Arguments: 
{
  "query": "CREATE TABLE Orders (order_id INT, customer_id INT, order_date DATE, total_amount DECIMAL(10,2), status VARCHAR(10));"
}
********************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION run_sql...
SnowSQLAgent (to chat_manager):

***** Response from calling function "run_sql" *****
Error: (1304): 01b01161-3203-054d-0000-0001486630a9: 002002 (42710): SQL compilation error:
Object 'ORDERS' already exists.
****************************************************

-----------------------------------------------------------------------

## Run a Join query - 1

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="""
    This dataset provides the current and historical technology usage for the top 10,000 websites on the internet according to the BuiltWith Top Site rank as well as other useful meta details about the websites in the list.

It lists all of the website names, technology names, categories, technology detected dates, estimated technology spend, estimated eCommerce revenue, BuiltWith Site Rank.

Tables Included :
- WebsiteMeta
- WebsiteTechnology

Fields included in WEBSITE_META table:
- Domain Name
- First Indexed Date
- Last Indexed Date
- Estimated USD Technology Spend per Month
- Estimated USD Sales Revenue per Month (for eCommerce sites)
- City
- Zip/State
- Country
- Company Name


Fields included in WEBSITE_TECH table:
- Domain Name
- Location on Site (where technology detected)
- Technology Name
- First Detected Date
- Last Detected Date
- Technology Major Category (i.e. analytics)
- Technology Minor Categories (i.e. A/B Testing)
- Technology Website
- IsPremium (yes or no if the technology is paid for - i.e. not free to use)


Using the above tables. Find the list of the most popular premium technologies found on sites that have a technology spend over $1000 a month.
""",
)

Admin (to chat_manager):

 
    This dataset provides the current and historical technology usage for the top 10,000 websites on the internet according to the BuiltWith Top Site rank as well as other useful meta details about the websites in the list.

It lists all of the website names, technology names, categories, technology detected dates, estimated technology spend, estimated eCommerce revenue, BuiltWith Site Rank.

Tables Included :
- WebsiteMeta
- WebsiteTechnology

Fields included in WEBSITE_META table:
- Domain Name
- First Indexed Date
- Last Indexed Date
- Estimated USD Technology Spend per Month
- Estimated USD Sales Revenue per Month (for eCommerce sites)
- City
- Zip/State
- Country
- Company Name


Fields included in WEBSITE_TECH table:
- Domain Name
- Location on Site (where technology detected)
- Technology Name
- First Detected Date
- Last Detected Date
- Technology Major Category (i.e. analytics)
- Technology Minor Categories (i.e. A/B Testing)
- Technology Website
- 

Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


## Run a join query - 2

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="""
This dataset provides the current and historical technology usage for the top 10,000 websites on the internet according to the BuiltWith Top Site rank as well as other useful meta details about the websites in the list.

It lists all of the website names, technology names, categories, technology detected dates, estimated technology spend, estimated eCommerce revenue, BuiltWith Site Rank.

Tables Included :
- Website_Meta
- Website_Tech

Fields included in WEBSITE_META table:
- Domain
- First Indexed Date - FIRST_INDEXED
- Last Indexed Date - LAST_INDEXED
- Estimated USD Technology Spend per Month - TECH_SPEND
- Estimated USD Sales Revenue per Month (for eCommerce sites) - SALES_REVENUE
- City -  CITY
- Zip/State - ZIP
- Country - COUNTRY_ISO
- Company_Name


Fields included in WEBSITE_TECH table:
- Domain
- Location on Site (where technology detected) - LOCATION_ON_SITE
- Technology Name - TECH_NAME
- First Detected Date - FIRST_DETECTED
- Last Detected Date - LAST_DETECTED
- Technology Major Category (i.e. analytics) - MAJOR_CATEGORY
- Technology Minor Categories (i.e. A/B Testing) - MINOR_CATEGORIES
- Technology Website - TECH_WEBSITE
- IsPremium (yes or no if the technology is paid for - i.e. not free to use) - IS_PREMIUM


With the above context, Find the Domains and their Total Revenue where the last detected technology is Twitter Analytics. Order it by total revenue in descending order.
""",
)

Admin (to chat_manager):

 
This dataset provides the current and historical technology usage for the top 10,000 websites on the internet according to the BuiltWith Top Site rank as well as other useful meta details about the websites in the list.

It lists all of the website names, technology names, categories, technology detected dates, estimated technology spend, estimated eCommerce revenue, BuiltWith Site Rank.

Tables Included :
- Website_Meta
- Website_Tech

Fields included in WEBSITE_META table:
- Domain
- First Indexed Date - FIRST_INDEXED
- Last Indexed Date - LAST_INDEXED
- Estimated USD Technology Spend per Month - TECH_SPEND
- Estimated USD Sales Revenue per Month (for eCommerce sites) - SALES_REVENUE
- City -  CITY
- Zip/State - ZIP
- Country - COUNTRY_ISO
- Company_Name


Fields included in WEBSITE_TECH table:
- Domain
- Location on Site (where technology detected) - LOCATION_ON_SITE
- Technology Name - TECH_NAME
- First Detected Date - FIRST_DETECTED
- Last Detected Date

Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


## CDC implementation ?

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="""
Fields included in WEBSITE_META table:
- Domain Name
- First Indexed Date
- Last Indexed Date
- Estimated USD Technology Spend per Month
- Estimated USD Sales Revenue per Month (for eCommerce sites)
- City
- Zip/State
- Country
- Company Name

With the above context. Design and implement CDC approach to store all the changes happening on the WEBSITE_META table.
Assume no other object is created other than the above tables.
ALWAYS Use the qualified name when referencing ALL objects i.e, use the DEMO_DB.DEMO_SCH.OBJECT_NAME.
Test it and provide the final approach implemented. You can use the COMPUTE_WH warehouse for all your needs.
""",
)

Admin (to chat_manager):

 
Fields included in WEBSITE_META table:
- Domain Name
- First Indexed Date
- Last Indexed Date
- Estimated USD Technology Spend per Month
- Estimated USD Sales Revenue per Month (for eCommerce sites)
- City
- Zip/State
- Country
- Company Name

With the above context. Design and implement CDC approach to store all the changes happening on the WEBSITE_META table. 
Assume no other object is created other than the above tables. 
ALWAYS Use the qualified name when referencing ALL objects i.e, use the DEMO_DB.DEMO_SCH.OBJECT_NAME.
Test it and provide the final approach implemented. You can use the COMPUTE_WH warehouse for all your needs.


--------------------------------------------------------------------------------
SnowEngineer (to chat_manager):

To implement a CDC (Change Data Capture) approach for the WEBSITE_META table, we can use Snowflake's Stream and Task features. Here is the step-by-step approach:

1. Create a stream to capture the changes happening o

Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  SQL compilation error: error line 1 at position 89 invalid identifier 'METADATA$VERSION'


Admin (to chat_manager):

SQL compilation error: error line 1 at position 89 invalid identifier 'METADATA$VERSION'

--------------------------------------------------------------------------------
SnowEngineer (to chat_manager):

I apologize for the mistake. The correct identifier for the version column in the stream metadata is `METADATA$STREAM_VERSION`, not `METADATA$VERSION`. 

Please modify the task query as follows:

```sql
-- Create a task
CREATE OR REPLACE TASK DEMO_DB.DEMO_SCH.WEBSITE_META_CDC_TASK
  WAREHOUSE = COMPUTE_WH
  SCHEDULE = '1 MINUTE'
AS
  -- Insert the changes into a CDC table
  INSERT INTO DEMO_DB.DEMO_SCH.CDC_WEBSITE_META
  SELECT METADATA$ACTION,
         METADATA$ISUPDATE,
         METADATA$ROW_ID,
         METADATA$STREAM_VERSION,
         CURRENT_TIMESTAMP(),
         DOMAIN_NAME,
         FIRST_INDEXED_DATE,
         LAST_INDEXED_DATE,
         ESTIMATED_USD_TECHNOLOGY_SPEND,
         ESTIMATED_USD_SALES_REVENUE,
         CITY,
         ZIP_STATE,
         C